In [1]:
import numpy as np
import pandas as pd

DATA_DIR = "data"

In [2]:
y = pd.read_csv(DATA_DIR + '/submission_format.csv', index_col='building_id')

y_top = pd.read_csv(DATA_DIR + '/submission_7533.csv', index_col='building_id')

cat_1 = np.load(DATA_DIR + '/intermediate/catboost_5fold_1_2_test.npy')
cat_3 = np.load(DATA_DIR + '/intermediate/catboost_gscv_2_3_test.npy')

gbm_1 = np.load(DATA_DIR + '/intermediate/lightgbm_5fold_1_2_test.npy')
gbm_3 = np.load(DATA_DIR + '/intermediate/lightgbm_5fold_2_3_test.npy')

In [3]:
# cat_1_df = pd.DataFrame(data=cat_1, columns=["2","1"], index=y.index)
# gbm_1_df = pd.DataFrame(data=gbm_1, columns=["1"], index=y.index)
# gbm_1_df["2"] = 1 - gbm_1_df["1"]

cat_1_df = pd.DataFrame(data=cat_1, columns=["1","2"], index=y.index)
gbm_1_df = pd.DataFrame(data=gbm_1, columns=["2"], index=y.index)
gbm_1_df["1"] = 1 - gbm_1_df["2"]

cat_3_df = pd.DataFrame(data=cat_3, columns=["2","3"], index=y.index)
gbm_3_df = pd.DataFrame(data=gbm_3, columns=["3"], index=y.index)
gbm_3_df["2"] = 1 - gbm_3_df["3"]

ensemble_1 = cat_1_df.add(gbm_1_df)
ensemble_3 = cat_3_df.multiply(1.2).add(gbm_3_df)

In [4]:
conflicts = 0
changes = {"1":{},"2":{},"3":{}}

y_pred = y_top

for index, row in y_top.iterrows():
    pred_3 = str(row["damage_grade"])
    pred_1_2 = ensemble_1.loc[index].idxmax()
    pred_2_3 = ensemble_3.loc[index].idxmax()
    pred = pred_3
    if(pred_3 == "1"):
        pred = pred_1_2
    elif(pred_3 == "3"):
        pred = pred_2_3
    else:
        if(pred_1_2 == "1"):
            if(pred_2_3 == "3"):
                conflicts += 1
            else:
                pred = "1"
        else:
            pred = pred_2_3
    if(pred in changes[pred_3]):
        changes[pred_3][pred] += 1
    else:
        changes[pred_3][pred] = 1
    y_pred.loc[index,"damage_grade"]=pred

print(conflicts)
print(changes)

y_pred

2
{'1': {'1': 6421, '2': 356}, '2': {'2': 54700, '3': 678, '1': 251}, '3': {'3': 23867, '2': 595}}


,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3
...,...
310028,2
663567,3
1049160,2


In [200]:
y_old = pd.read_csv(DATA_DIR + '/submission_try.csv', index_col='building_id')
diff = y_old[y_old["damage_grade"]!=y_pred["damage_grade"].astype(int)]
diff = diff.join(y_pred,lsuffix="_old",rsuffix="_new",how="left")
diff

,damage_grade_old,damage_grade_new
building_id,,
924792,3,2
984386,2,3
1043856,3,2
918497,2,3
149515,2,3
761223,2,3
330385,3,2
69823,2,3
554867,3,2


In [199]:
y_std = pd.read_csv(DATA_DIR + '/submission_7541.csv', index_col='building_id')

cnt = 0
for index,row in diff.iterrows():
    if(str(y_std.loc[index,"damage_grade"])==str(row["damage_grade_old"])):
        y_std.loc[index,"damage_grade"]=str(row["damage_grade_new"])

y_std.to_csv(DATA_DIR+'/submission_try.csv')

In [197]:
y_pred.to_csv(DATA_DIR + '/submission_120.csv')